# Setting up

In [1]:
folder_30days = 'C:\\Users\\Pleng Weerada\\Documents\\MSc Project\\flight_data\\'
folder_60days = 'C:\\Users\\Pleng Weerada\\Documents\\MSc Project\\flight_data_60days\\'
chrome_file = r"C:\\Users\\Pleng Weerada\\Downloads\\chrome-win64_116\\chrome-win64\\chrome.exe"
chrome_driver_file = r"C:\\Users\\Pleng Weerada\\Downloads\\chromedriver-win64_116\\chromedriver-win64\\chromedriver.exe"

In [2]:
# pip install jupyter_scheduler

In [3]:
## install require package by uncomment below command
# !pip install selenium
# !pip install webdriver_manager
# !pip install beautifulsoup4

In [4]:
from selenium import webdriver

from selenium.webdriver.chrome.service import Service

from selenium.webdriver.support.ui import WebDriverWait

from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.chrome.options import Options

from bs4 import BeautifulSoup

import codecs #write to a text file

import re #regular expression

from webdriver_manager.chrome import ChromeDriverManager

import datetime #get search datetime

import csv #write serach result to csv

import time

import os

import pandas as pd #for generate depart date list

from datetime import timedelta, date

# Function

## get_html_pageSource

In [5]:
def get_html_pageSource(url):
    
#     driver=webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    
    wait = WebDriverWait(driver, 20)
    
    print('Getting html...')

    driver.get(url)

    get_url = driver.current_url
    
    wait.until(EC.url_to_be(url))

    if get_url == url:
        page_source = driver.page_source
        
    time.sleep(30)
    
    print('Success getting html for ', url)
    
#     driver.close()
        
    return page_source

## create_csv_flight_detail_from_html

In [6]:
def create_csv_flight_detail_from_html(soup, file_name):
    # get data from beautifulsoup html object, and create data row for csv file
    route = soup.title.text.split(",")[0]
    depart_dt = soup.title.text.split(",")[1]
    search_dt = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    price_lst = soup.find_all("div", {"class": "f8F1-price-text"})
    airline_list = soup.find_all("div", {"class": "J0g6-operator-text"})
    direct_list = soup.find_all("span", {"class": "JWEO-stops-text"})

    data = []
    for i in range(0,len(price_lst)):
        col_price = re.search('(\<.+?\>)(.+?)(\<.+?\>)', str(price_lst[i])).group(2)
        col_airline = re.search('(\<.+?\>)(.+?)(\<.+?\>)', str(airline_list[i])).group(2)
        col_direct = re.search('(\<.+?\>)(.+?)(\<.+?\>)', str(direct_list[i])).group(2)
        row = [route, depart_dt, search_dt, col_price, col_airline, col_direct]
        data.append(row)
    
    # create csv file
    header = ['route' ,'depart_dt' ,'search_dt' ,'ticket_price' ,'airline_nm' ,'direct']
    with open(file_name, 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)

        # write the header
        writer.writerow(header)

        # write multiple rows
        writer.writerows(data)

    print("Success create file: " + file_name)

## generate_url_list_from_dt

In [7]:
def generate_url_list_from_dt(start_dt, end_dt):
    url_rerun = []
    url_depart_dt = pd.date_range(start_dt, end_dt,freq='D')
    for dt in url_depart_dt:
        str_dt = str(dt)[:10]
        urls = [
         'https://www.kayak.co.uk/flights/LON-BKK/'+str_dt+'?sort=bestflight_a'
        ,'https://www.kayak.co.uk/flights/LON-ICN/'+str_dt+'?sort=bestflight_a'
        ,'https://www.kayak.co.uk/flights/LON-HND/'+str_dt+'?sort=bestflight_a'
        ,'https://www.kayak.co.uk/flights/LON-HKG/'+str_dt+'?sort=bestflight_a'
        ,'https://www.kayak.co.uk/flights/LON-SIN/'+str_dt+'?sort=bestflight_a'
        ]
        url_rerun += urls
    return url_rerun

## scraping_from_list

In [8]:
def scraping_from_list(url_rerun, scarp60days):
#     driver=webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    print('='*30)
    print('length url_rerun {0}, url_rerun \n{1}'.format( len(url_rerun), url_rerun ))
    start_time = datetime.datetime.now()
    print('start_time ',start_time.strftime("%H:%M"))
    start_length_url = len(url_rerun)
    i = 0
    while len(url_rerun) > 0 and i < 11:
#     while len(url_rerun) > 0:
        for url in url_rerun:
            print('-'*30)
            print(url)
            # get html
            page_source = get_html_pageSource(url)
            page_source = get_html_pageSource(url)

            # get information from html
            soup = BeautifulSoup(page_source,features="html.parser")

            # create file_name
            dt = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
            route = soup.title.text.split(",")[0]
            dp_dt = soup.title.text.split(",")[1]
            folder = folder_30days #'C:\\Users\\Pleng Weerada\\Documents\\MSc Project\\flight_data\\'

            if scarp60days:
                folder = folder_60days #'C:\\Users\\Pleng Weerada\\Documents\\MSc Project\\flight_data_60days\\'

            file_name = folder \
                    + route.replace(" ", "-")\
                    + dp_dt.replace("/", "-").replace(" ", "_")\
                    +"_"+str(dt)+'.csv' 

            # create file flight data
            create_csv_flight_detail_from_html(soup, file_name)
            file_size = os.path.getsize(file_name)
            print('file size ', file_size)

            if(file_size >= 1020):
                url_rerun.remove(url)
            else:
                os.remove(file_name)
            print('len(url_rerun) ', len(url_rerun))
            print('url_rerun ', url_rerun)
        i+=1
        print('i {0}, remaining {1} urls'.format(i, len(url_rerun)))

    finish_time = datetime.datetime.now()
    print('{0}-{1} ({2})'.format(start_time.strftime("%H:%M")
                                 , finish_time.strftime("%H:%M")
                                 , start_length_url-len(url_rerun) ))
    
    return url_rerun

# 30 days prior to depart date

## 1

In [9]:
## ------ 30 days
## 1)
scarp60days = False
start_dt = date(2023, 9, 20)
end_dt = date(2023, 9, 21)
url_lst1 = generate_url_list_from_dt(start_dt, end_dt)

# for chrome version <= 114
# driver=webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# for chrome version 116
# taken from: 
# https://stackoverflow.com/questions/76913935/selenium-common-exceptions-sessionnotcreatedexception-this-version-of-chromedri
# https://stackoverflow.com/questions/29858752/error-message-chromedriver-executable-needs-to-be-available-in-the-path

options = webdriver.ChromeOptions()

# options.binary_location = r"C:\\Users\\Pleng Weerada\\Downloads\\chrome-win64_116\\chrome-win64\\chrome.exe"
# service_obj = Service(r"C:\\Users\\Pleng Weerada\\Downloads\\chromedriver-win64_116\\chromedriver-win64\\chromedriver.exe")
options.binary_location = chrome_file
service_obj = Service(chrome_driver_file)

driver = webdriver.Chrome(service=service_obj, options=options)

# Check missing URL
url_lst1_remain = scraping_from_list(url_lst1, scarp60days)
driver.close()

length url_rerun 10, url_rerun 
['https://www.kayak.co.uk/flights/LON-BKK/2023-09-20?sort=bestflight_a', 'https://www.kayak.co.uk/flights/LON-ICN/2023-09-20?sort=bestflight_a', 'https://www.kayak.co.uk/flights/LON-HND/2023-09-20?sort=bestflight_a', 'https://www.kayak.co.uk/flights/LON-HKG/2023-09-20?sort=bestflight_a', 'https://www.kayak.co.uk/flights/LON-SIN/2023-09-20?sort=bestflight_a', 'https://www.kayak.co.uk/flights/LON-BKK/2023-09-21?sort=bestflight_a', 'https://www.kayak.co.uk/flights/LON-ICN/2023-09-21?sort=bestflight_a', 'https://www.kayak.co.uk/flights/LON-HND/2023-09-21?sort=bestflight_a', 'https://www.kayak.co.uk/flights/LON-HKG/2023-09-21?sort=bestflight_a', 'https://www.kayak.co.uk/flights/LON-SIN/2023-09-21?sort=bestflight_a']
start_time  11:48
------------------------------
https://www.kayak.co.uk/flights/LON-BKK/2023-09-20?sort=bestflight_a
Getting html...
Success getting html for  https://www.kayak.co.uk/flights/LON-BKK/2023-09-20?sort=bestflight_a
Getting html...
Su

## 2

In [10]:
# 2)
## time.sleep(3600)
# scarp60days = False
# start_dt = date(2023, 9, 22)
# end_dt = date(2023, 9, 23)
# url_lst2 = generate_url_list_from_dt(start_dt, end_dt)

# driver=webdriver.Chrome(service=Service(ChromeDriverManager().install()))
# url_lst2_remain = scraping_from_list(url_lst2, scarp60days)
# driver.close()

# References

* https://www.browserstack.com/guide/web-scraping-using-selenium-python
* https://www.pythontutorial.net/python-basics/python-write-csv-file/